<a href="https://colab.research.google.com/github/deovrat-n/Fake-News-Classifier/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/MLPROJECTS/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
###Drop Nan Values
df=df.dropna()


In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.8.2'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1714, 414, 3337, 4334, 1267, 2851, 890, 459, 1265, 4229],
 [824, 662, 4726, 2227, 3531, 3607, 1646],
 [597, 4158, 1332, 2654],
 [1217, 1180, 2144, 4338, 3874, 780],
 [4346, 3531, 4205, 506, 4505, 2964, 3531, 3214, 2346, 1298],
 [2614,
  2562,
  1142,
  3540,
  1786,
  4299,
  2345,
  294,
  1353,
  3007,
  156,
  3894,
  4295,
  1137,
  1646],
 [4295, 892, 4182, 3411, 459, 3796, 3124, 464, 1635, 2172, 763],
 [1652, 4213, 620, 2731, 1526, 215, 4299, 2991, 1635, 2172, 763],
 [4266, 685, 2683, 4447, 4574, 3199, 4359, 130, 4299, 2068],
 [4389, 1145, 2017, 4751, 445, 4102, 4410, 2162],
 [2695, 2275, 3912, 3333, 773, 1672, 3900, 4373, 233, 2461, 216],
 [4338, 4807, 1267, 3199, 4299, 1526],
 [1773, 1877, 4855, 1429, 795, 329, 102, 4453, 4676],
 [4575, 2048, 4188, 2638, 519, 4915, 1140, 1635, 2172, 763],
 [3921, 4258, 192, 1607, 2582, 1635, 2172, 763],
 [2965, 1307, 2764, 1281, 4561, 1539, 4898, 4284, 1721, 2316],
 [4649, 3524, 662],
 [2940, 3320, 2284, 3930, 4299, 737, 4401, 1646],
 [1279, 

### Embedding Representation

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  459 1265 4229]
 [   0    0    0 ... 3531 3607 1646]
 [   0    0    0 ... 4158 1332 2654]
 ...
 [   0    0    0 ... 1635 2172  763]
 [   0    0    0 ... 2043 4881 4059]
 [   0    0    0 ...  377 2114 2265]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1714,
        414, 3337, 4334, 1267, 2851,  890,  459, 1265, 4229], dtype=int32)

In [25]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 37ms/step - loss: 0.3269 - accuracy: 0.8476 - val_loss: 0.1923 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1335 - accuracy: 0.9481 - val_loss: 0.2052 - val_accuracy: 0.9127
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0941 - accuracy: 0.9638 - val_loss: 0.2661 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0697 - accuracy: 0.9741 - val_loss: 0.2724 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0533 - accuracy: 0.9809 - val_loss: 0.3827 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0421 - accuracy: 0.9849 - val_loss: 0.3519 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0281 - accuracy: 0.9919 - val_loss: 0.4819 - val_accuracy: 0.912

### Adding Dropout 

In [31]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [49]:
y_pred=model.predict(X_test)
y_pred=np.round(y_pred).astype(int)

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[2769,  650],
       [ 825, 1791]])

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7555923777961889